<a href="https://colab.research.google.com/github/abdulahad89/Encoder-Decoder-based-Translation-Model/blob/main/SR_seq2seq_original.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install AudioSegment

In [ ]:
import pandas as pd
import os
import librosa
import IPython.display as ipd
import matplotlib.pyplot as plt
from scipy.io import wavfile
import warnings
import glob 
from pydub import AudioSegment

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv('drive/My Drive/Sajid_Project_Grading/SR/full_data.csv')

In [ ]:
aud_data = []
path_2 = 'drive/My Drive/Sajid_Project_Grading/SR/clips_wavfiles'
for wav in data['path']:
    samples , sample_rate = librosa.load(path_2 +'/'+ wav[:-4] + '.wav',16000) 
    samples = librosa.resample(samples, sample_rate, 8000)
    aud_data.append(samples)

In [ ]:
max_input_len = max(len(sen) for sen in aud_data)

In [ ]:
max_input_len

73728

In [ ]:
print(aud_data[25])

[ 0.0000000e+00  0.0000000e+00  0.0000000e+00 ... -3.6458505e-06
 -5.4983611e-06  9.1177344e-06]


In [ ]:
import os, sys
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, GRU, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
sen_len = data['sentence'].apply(lambda x: len(x.split(' ')))
max(sen_len)

12

In [ ]:
BATCH_SIZE = 32
EPOCHS = 8
LSTM_NODES =256
NUM_SENTENCES = 161
MAX_SENTENCE_LENGTH = max(sen_len)
MAX_NUM_WORDS = 5000
EMBEDDING_SIZE = 100


In [ ]:
output_sentences = []
output_sentences_inputs = []

count = 0
for i in range(161):
    count += 1

    if count > NUM_SENTENCES:
        break

    output = data.iloc[i,3]

    output_sentence = output + ' <eos>'
    output_sentence_input = '<sos> ' + output

    output_sentences.append(output_sentence)
    output_sentences_inputs.append(output_sentence_input)


In [ ]:
#Tokenizing output sequences and 1 palce shifted o/p sequences as an input to o/p 
output_tokenizer = Tokenizer(num_words=MAX_NUM_WORDS, filters='')
output_tokenizer.fit_on_texts(output_sentences + output_sentences_inputs)
output_integer_seq = output_tokenizer.texts_to_sequences(output_sentences)
output_input_integer_seq = output_tokenizer.texts_to_sequences(output_sentences_inputs)

word2idx_outputs = output_tokenizer.word_index

num_words_output = len(word2idx_outputs) + 1
max_out_len = max(len(sen) for sen in output_integer_seq)

In [ ]:
output_tokenizer.word_index

{'<eos>': 1,
 '<sos>': 2,
 'ਅਕਾਲ': 519,
 'ਅਕਾਲੀਆਂ': 367,
 'ਅਖਿਆ': 327,
 'ਅਗਰ': 376,
 'ਅਗੰਮੜੇ': 550,
 'ਅਜੇ': 611,
 'ਅਤੇ': 57,
 'ਅਧਿਕਾਰ': 287,
 'ਅਨਮੋਲ': 410,
 'ਅਫ਼ਸੋਸ': 388,
 'ਅਭਿਨਵ': 691,
 'ਅਭੁੱਲ': 526,
 'ਅਮਰੀਕਾ': 453,
 'ਅਮੀਰ': 495,
 'ਅਮੀਰੀ': 196,
 'ਅਸਤਿਤੱਵ': 415,
 'ਅਸਲੀਅਤ': 103,
 'ਅਸੀਂ': 52,
 'ਅੜੀਅਲ': 573,
 'ਅੰਗਰੇਜ਼ੀ': 261,
 'ਅੰਦਰ': 101,
 'ਅੰਦਾਜ਼': 571,
 'ਅੰਮ੍ਰਿਤ': 311,
 'ਅੱਖੀਆਂ': 677,
 'ਅੱਗੇ': 106,
 'ਅੱਜ': 41,
 'ਅੱਟੀ': 577,
 'ਆਂਵਦਾ': 683,
 'ਆਇਆ': 63,
 'ਆਈ': 71,
 'ਆਕਸੀਜਨ': 305,
 'ਆਖ': 520,
 'ਆਖਿਆ': 323,
 'ਆਤਮਾ': 331,
 'ਆਦੀ': 259,
 'ਆਪ': 253,
 'ਆਪਣਾ': 68,
 'ਆਪਣੀ': 129,
 'ਆਪਣੇ': 64,
 'ਆਫ਼ਰਿਆ': 633,
 'ਆਮ': 605,
 'ਆਸ਼੍ਰਮ': 477,
 'ਇਕ': 107,
 'ਇਕੱਠ': 176,
 'ਇਥੇ': 609,
 'ਇਨ੍ਹਾਂ': 506,
 'ਇਸ': 14,
 'ਇਹ': 18,
 'ਇਹਨੂੰ': 255,
 'ਇਹੀ': 527,
 'ਇੱਕ': 33,
 'ਈ': 135,
 'ਉਕਤ': 479,
 'ਉਣੱਤਰਵੇਂ': 208,
 'ਉਦੋਂ': 602,
 'ਉਨਾਂ': 383,
 'ਉਨ੍ਹਾਂ': 126,
 'ਉਮਰ': 123,
 'ਉਮਰੇ': 658,
 'ਉਸ': 31,
 'ਉਸਨੂੰ': 617,
 'ਉਸੇ': 81,
 'ਉਹ': 17,
 'ਉਹਦੀ': 121,
 'ਉਹਦੇ': 118,
 'ਉਹਨਾਂ': 125,
 'ਉੱਚ': 551,
 'ਉੱਚਾ': 230,
 'ਏ': 148,
 'ਏਨੀ': 18

In [ ]:
#Padding of i/p, o/p, i/p to o/p sequences
encoder_input_sequences = pad_sequences(aud_data, maxlen=max_input_len)
decoder_input_sequences = pad_sequences(output_input_integer_seq, maxlen=max_out_len, padding='post')
decoder_output_sequences = pad_sequences(output_integer_seq, maxlen=max_out_len, padding='post')

In [ ]:
encoder_input_sequences = encoder_input_sequences.reshape(161,73728,1)

In [ ]:
encoder_input_sequences.shape

(161, 73728)

In [ ]:
decoder_targets_one_hot = np.zeros((
        161,
        max_out_len,
        num_words_output
    )
)

In [ ]:
decoder_targets_one_hot.shape

for i, d in enumerate(decoder_output_sequences):
    for t, word in enumerate(d):
        decoder_targets_one_hot[i, t, word] = 1

In [ ]:
#encoder_inputs_placeholder = Input(shape=(161,1,max_input_len,))

In [ ]:
#encoder_inputs_placeholder.shape

TensorShape([None, 161, 1, 73728])

In [ ]:
#Encoder part      
encoder_inputs_placeholder = Input(shape=(None,1))
#x = embedding_layer(encoder_inputs_placeholder)
encoder = LSTM(LSTM_NODES,return_state=True)

#encoder states
encoder_outputs, h, c = encoder(encoder_inputs_placeholder)
encoder_states = [h, c]


In [ ]:
encoder_inputs_placeholder.shape

TensorShape([None, 73728, 1])

In [ ]:
#Decoder part
decoder_inputs_placeholder = Input(shape=(max_out_len,))
decoder_embedding = Embedding(num_words_output, LSTM_NODES)
decoder_inputs_x = decoder_embedding(decoder_inputs_placeholder)
decoder_lstm = LSTM(LSTM_NODES, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(decoder_inputs_x, initial_state=encoder_states)


In [ ]:
#Dense layer for softmax (final layer)
decoder_dense = Dense(num_words_output, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs_placeholder,
  decoder_inputs_placeholder], decoder_outputs)
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)

In [ ]:
r = model.fit(
    [encoder_input_sequences, decoder_input_sequences],
    decoder_targets_one_hot,
    batch_size=BATCH_SIZE,
    epochs=8,
    validation_split=0.1
)
    

/usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3350: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


Epoch 1/8


InternalError: ignored

In [ ]:
encoder_model = Model(encoder_inputs_placeholder, encoder_states)

decoder_state_input_h = Input(shape=(LSTM_NODES,))
decoder_state_input_c = Input(shape=(LSTM_NODES,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_inputs_single = Input(shape=(1,))
decoder_inputs_single_x = decoder_embedding(decoder_inputs_single)

decoder_outputs, h, c = decoder_lstm(decoder_inputs_single_x, initial_state=decoder_states_inputs)

decoder_states = [h, c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs_single] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

In [ ]:
#idx2word_input = {v:k for k, v in word2idx_inputs.items()}
idx2word_target = {v:k for k, v in word2idx_outputs.items()}

In [ ]:
def translate_sentence(input_seq):
    states_value = encoder_model.predict(input_seq)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = word2idx_outputs['<sos>']
    eos = word2idx_outputs['<eos>']
    output_sentence = []

    for _ in range(max_out_len):
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
        idx = np.argmax(output_tokens[0, 0, :])

        if eos == idx:
            break

        word = ''

        if idx > 0:
            word = idx2word_target[idx]
            output_sentence.append(word)

        target_seq[0, 0] = idx
        states_value = [h, c]

    return ' '.join(output_sentence)


In [ ]:
states_value = encoder_model.predict(encoder_input_sequences[3].reshape(1,1,73728))
states_value

/usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3350: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


[array([[ 1.9832416e-03, -2.9392836e-03, -2.7708495e-03, -2.3915293e-03,
         -1.3431433e-03, -2.0549148e-03,  1.9918962e-03,  1.8797313e-03,
         -1.4383299e-03,  3.0026499e-03,  1.4180391e-03, -1.8441672e-03,
         -1.6917244e-03, -6.3298000e-03, -2.6564049e-03,  3.1166405e-03,
          6.9447380e-04,  2.3725205e-03,  2.9290322e-04,  3.3212297e-03,
          5.5863668e-04, -1.3160881e-03,  2.5751279e-03, -9.5557328e-04,
          3.9126314e-03, -2.9484942e-03,  1.6063093e-03,  1.7228012e-03,
         -2.3650743e-04,  1.0464938e-03,  2.0690663e-03, -4.1803614e-05,
         -2.0837293e-03,  3.1804154e-03, -9.3866533e-05, -1.2077554e-03,
         -2.3056786e-03,  3.2852050e-03,  2.9658568e-03,  2.0721594e-03,
         -2.4969603e-03, -1.5396343e-03, -2.2479810e-03,  6.6872533e-05,
         -2.8486291e-03,  3.3057588e-03, -2.6770008e-03, -3.2109372e-05,
          1.0115480e-03,  2.3922769e-03, -1.0419845e-03,  2.3755596e-03,
         -5.7630130e-04,  6.3022750e-04, -3.1923240

In [ ]:
target_seq = np.zeros((1, 1))
target_seq[0, 0] = word2idx_outputs['<sos>']

In [ ]:
len([target_seq]+states_value)

3

In [ ]:
output_tokens, h, c = decoder_model.predict([target_seq] + states_value)
idx = np.argmax(output_tokens[0, 0, :])

/usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3350: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


In [ ]:
output_tokens[0,0,:]

array([0.00172956, 0.00177001, 0.0012265 , 0.00176879, 0.00179913,
       0.00179007, 0.00172425, 0.00175066, 0.00173131, 0.00167894,
       0.00173386, 0.0017218 , 0.00169601, 0.00171647, 0.00174108,
       0.00172376, 0.0017106 , 0.00170398, 0.00168645, 0.0016597 ,
       0.0016996 , 0.00164338, 0.00166485, 0.00166291, 0.00164997,
       0.00166033, 0.00165538, 0.00162656, 0.00162956, 0.00164341,
       0.00166859, 0.00157233, 0.0016225 , 0.00164828, 0.00160538,
       0.00156369, 0.00164907, 0.00163975, 0.00160102, 0.00160802,
       0.00157536, 0.00156258, 0.00161845, 0.00157029, 0.00159541,
       0.00158064, 0.00157209, 0.00152727, 0.00154338, 0.00160316,
       0.00153571, 0.00158131, 0.00144898, 0.00157023, 0.00150381,
       0.00149521, 0.00152402, 0.00155713, 0.00147589, 0.00152981,
       0.00131439, 0.00150752, 0.00151958, 0.00151893, 0.00144352,
       0.00152895, 0.00146882, 0.00148422, 0.00151166, 0.00150032,
       0.00153442, 0.00151028, 0.00143375, 0.0015163 , 0.00145

In [ ]:
idx2word_target[4]

'ਦੀ'

In [ ]:
import pandas as pd
pd.options.display.max_rows = 4000

In [ ]:
#|TESTIMG MODEL
i = np.random.choice(len(aud_data))
input_seq = encoder_input_sequences[i]
#print(input_seq.shape)
input_seq = input_seq.reshape(1,1,73728)
text = translate_sentence(input_seq)
print('-')
print(i)
#print('Input:', input_sentences[i])
print('Response:', text)


/usr/local/lib/python3.6/dist-packages/tensorflow/python/data/ops/dataset_ops.py:3350: UserWarning: Even though the tf.config.experimental_run_functions_eagerly option is set, this option does not apply to tf.data functions. tf.data functions are still traced and executed as graphs.
  "Even though the tf.config.experimental_run_functions_eagerly "


-
147
Response: 


In [ ]:
data.iloc[i,3]

'ਕੁੱਝ ਪੈਸਾ ਇਸ ਕੰਮ ਵਾਸਤੇ ਵੀ ਰੱਖੋ'